### Correlation plots

Read counts in overlapping peaks (DRIPc, ChIP, RNA-seq)

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from scipy.stats import pearsonr

In [ ]:
# Make SVG text as font not as curves
mpl.rcParams['svg.fonttype'] = 'none'

mpl.rcParams['font.sans-serif'].insert(0, 'Arial')
mpl.rcParams['font.family'] = 'sans-serif'

In [ ]:
# If one doesn't want to save, every time all the cells are run, set to False
SAVE_FIGS = True

In [ ]:
# Path to a directory with bam stats files (generated with `samtools stats`)
SRC_DIR_PATH = Path(r"/path/to/data/bam")

DEST_DIR_PATH = Path(r"/path/to/data/bam")

##### Get the number of mapped and paired reads (fragments) from '.stats' files and calculate size factors.
One can generate a dict with the size factors of all libraries handled in the project.

In [ ]:
def get_mapped_reads_count(file_name: str) -> int:
    with open(file_name, mode='r') as f:
        for line in f.readlines():
            if line.startswith('#'):
                continue
            elif line.startswith('SN'):
                data = line.split('\t')
                if data[1].strip() == 'reads mapped and paired:':
                    return int(data[2].strip())

Generate a dictionary with sample names and library sizes.

The sample name is derived from the .stat file name.

In [ ]:
mapped_reads = {}
for file_path in SRC_DIR_PATH.glob("*.stats"):
    file_name = file_path.stem
    reads_count = get_mapped_reads_count(file_path)
    print(f"{file_name}: {reads_count}")
    mapped_reads[file_name] = reads_count

Get size factors

In [ ]:
size_factors = {}
for sample, reads in mapped_reads.items():
    size_factors[sample_name] = reads/1e6

In [ ]:
# size_factors

Now we have a dictionary with sample file name and the library size factors.

In [ ]:
chromosomes = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 
               'chr7', 'chr8', 'chr9', 'chrM', 'chrX', 'chrY', 
               'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 
               'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 
               'chr20', 'chr21', 'chr22']

##### Read in peaks+reads file

In [ ]:
# Path to a file (bed, narrowPeak, bradPeak, other) with read counts in peaks/regions (rows)
# The 'chrom', 'chromStart', 'chromEnd' columns are mandatory.
# There should be named columns with samples.
# Example column names.
# example_col_names = [
#     'chrom', 'chromStart', 'chromEnd', 'name',
#     'score', 'strand', 'signalValue', 'pValue', 
#     'qValue', 'Dataset_1_1', 'Dataset_1_2', 'Dataset_2_1',
#     'Dataset_2_2', 'Dataset_3_1', 'Dataset_3_2'
# ]

peaks_file_path = Path(r"/path/to/peaks/counted/Dataset_1-2-3_overlap.txt")
peaks_data = pd.read_csv(peaks_file_path, sep='\t', decimal='.', header=None)

In [ ]:
peaks_data.head(3)

In [ ]:
peaks_data.describe()

Filter out contigs and non-standrd chromosomes.

In [ ]:
peaks_data_filtered = peaks_data.loc[peaks_data['chrom'].isin(chromosomes)].copy(deep=True)

Calculate peak width for FPKM calculation 

In [ ]:
peaks_data_filtered['peakLength'] = peaks_data_filtered['chromEnd'] - peaks_data_filtered['chromStart']

We need to log transform the data for correlations. In some peaks there maybe no reads (0).
So, one can add 1 to each peak fragments counts.

In [ ]:
peaks_data_filtered['Dataset_1_1'] = peaks_data_filtered['Dataset_1_1']+1
peaks_data_filtered['Dataset_1_2'] = peaks_data_filtered['Dataset_1_2']+1
peaks_data_filtered['Dataset_2_1'] = peaks_data_filtered['Dataset_2_1']+1
# And so on...

#### Normlisation

In [ ]:
columns_to_normalize = [
    'Dataset_1_1', 'Dataset_1_2', 
    'Dataset_2_1', 'Dataset_2_2',
    'Dataset_3_1', 'Dataset_3_2',
]

Normalize fragment counts with size factors.

In [ ]:
for col_name in columns_to_normalize:
    new_col_name = col_name+'_FPM'
    peaks_data_filtered[new_col_name] = (peaks_data_filtered[col_name]+1)/size_factors[col_name]

Normalize for peak length

In [ ]:
for col_name in columns_to_normalize:
    FPM_col_name = col_name+'_FPM'
    new_col_name = col_name+'_FPKM'
    peaks_data_filtered[new_col_name] = (peaks_data_filtered[FPM_col_name]+1)/(peaks_data_filtered['peakLength']/1000)

#### Average replicates

Generate means of replicate samples, if any. Here we just prepare dictionary with a mapping of replicates to a sample.

In [ ]:
samples_mapping = {
    "Dataset_1": ['Dataset_1_1', 'Dataset_1_2'],
    "Dataset_2": ['Dataset_2_1', 'Dataset_2_2'],
    "Dataset_3": ['Dataset_3_1', 'Dataset_3_2'],
    # And so on. One may have more replicates mapping to a final sample.
    # Keep in mind that the names in the lists must correspond to the column
    # names in the dataframe.
}

In [ ]:
for sample_name, samples in samples_mapping.items():
    peaks_data_filtered[sample_name] = peaks_data_filtered[samples].mean(axis=1)

In [ ]:
for sample_name, samples in samples_mapping_FPM.items():
    peaks_data_filtered[sample_name] = peaks_data_filtered[samples].mean(axis=1)

In [ ]:
for sample_name, samples in samples_mapping_FPKM.items():
    peaks_data_filtered[sample_name] = peaks_data_filtered[samples].mean(axis=1)

In [ ]:
peaks_data_filtered.describe()

In [ ]:
peaks_data_filtered.sample(5)

#### Pearson correlation

In [ ]:
pearson_r, pearson_pvalue = pearsonr(x=np.log10((peaks_data_filtered['Dataset_1']+1)), 
                                     y=np.log10((peaks_data_filtered['Dataset_2']+1)))

In [ ]:
pearson_r_FPM, pearson_pvalue_FPM = pearsonr(x=np.log10(peaks_data_filtered['Dataset_1_FPM']), 
                                     y=np.log10(peaks_data_filtered['Dataset_2_FPM']))

In [ ]:
pearson_r_FPKM, pearson_pvalue_FPKM = pearsonr(x=np.log10(peaks_data_filtered['Dataset_1_FPKM']), 
                                     y=np.log10(peaks_data_filtered['Dataset_2_FPKM']))

In [ ]:
g = sns.JointGrid(x=np.log10(peaks_data_filtered['Dataset_1_FPKM']), 
                  y=np.log10(peaks_data_filtered['Dataset_2_FPKM']), 
                  height=3.5)
g.plot_joint(sns.regplot, order=1, ci=95, fit_reg=False,
             scatter_kws={'s': 4.5, 'linewidths': 0.1, 
                                                'edgecolor': 'white'})
x0, x1 = g.ax_joint.get_xlim()
y0, y1 = g.ax_joint.get_ylim()
g.ax_joint.text(x0, y1-0.5, f'r={pearson_r_FPKM:.2f}\np-value={pearson_pvalue_FPKM:.2f}')
g.plot_marginals(sns.kdeplot, fill=True)
g.set_axis_labels('log10(Dataset 1 FPKM)', 'log10(Dataset 2 FPKM)')

In [ ]:
if SAVE_FIGS:
    fig_file_name = r"Dataset_1_vs_Dataset_2_correlation_jointplot"
    g.savefig(DEST_DIR_PATH.joinpath(fig_file_name+".png"), format='png', dpi=300, transparent=True)
    g.savefig(DEST_DIR_PATH.joinpath(fig_file_name+".pdf"), format='pdf', dpi=300, transparent=True)

Repeat the above steps for each pair of samples you have in your dataframe (from Pearson correlation heading)